<a href="https://colab.research.google.com/github/GTRe5/AI/blob/main/523H0135_lec06_LocalSearch_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Students implement Hill-Climbing Search, Local Beam Search, and Simulated Annealing Search algorithms following TODO 1 - 3. \
Students can add supporting attributes and methods to the three classes as needed.

# Libraries

In [27]:
import os
import heapq

# Graph class

In [28]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0
    self.H = dict()

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    for u, h in self.H.items():
      line = 'h(%d) = %d\n'%(u, h)
      res += line
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...
            u1 h1
            u2 h2
            u3 h3
            ...

        # input.txt
        7 8
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
        0 14
        1 13
        2 12
        3 11
        4 10
        5 9
        6 0
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for i in range(self.E):
          line = g.readline()
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))
        for i in range(self.V):
          line = g.readline()
          u, h = [int(it) for it in line.strip().split()]
          self.H[u] = h
  def get_neighbors(self, u):
    return self.AL.get(u, [])

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]
h(0) = 14
h(1) = 13
h(2) = 12
h(3) = 11
h(4) = 10
h(5) = 9
h(6) = 0



# Search Strategies

In [31]:
class LocalSearchStrategy:
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u
    '''
    return (None, None)

In [32]:
class HillClimbingSearch(LocalSearchStrategy):
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note: weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    '''
    # TODO 1
    u = src # Incase if src was the local maximum
    cost = 0 # G(n) with n = 0 - initial state
    p = cost + g.H[u] # H(n) + G(n)

    while True:
      neighbors = g.get_neighbors(u) # Get all neighbors
      if not neighbors:
        return (u, p) # Return if no neighbors

      improved = False # Track if we find a better move

      for neighbor, neighbor_cost in neighbors :
        generated_p = (neighbor_cost + cost) + g.H[neighbor]

        if generated_p > p:
          u = neighbor
          p = generated_p
          cost += neighbor_cost
          improved = True
          break

      if not improved:
        return (u, p)
    #   u, p = best_neighbor, best_p
    #   cost += best_cost
    # return (None, None)

In [33]:
class LocalBeamSearch(LocalSearchStrategy):
  def __init__(self, n = 5):  # Default beam width = 5
    self.n = n

  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which:
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note:
    - weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    - parameter n is provided in the constructor
    '''
    # u = src
    # cost = 0
    # p = cost + g.H[u]

    # beam = [(u, p)]  # Store (state, priority)
    states = [(0 + g.H[src], 0, src)]

    while states:
      next_states = []

      # Expand neighbors for each state in the beam
      for _, path_cost, u in states:
        for neighbor, cost in g.get_neighbors(u):
          new_cost = path_cost + cost
          new_priority = new_cost + g.H[neighbor]
          heapq.heappush(next_states, (new_priority, new_cost, neighbor))

      if not next_states:
        best_state = max(states, key = lambda x: x[0])  # Get the best found state
        return (best_state[2], best_state[0])  # Return best found state (u, p)

      # Select the top `k` best candidates
      states = heapq.nsmallest(self.n, next_states)

    return (None, None)


In [34]:
import math
import random

class SimulatedAnnealingSearch(LocalSearchStrategy):
  def schedule(self, t):
    return 1 / math.pow(t, 2)  # Cooling function
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which:
      u: the final state found
      p: the priority/weight/desirability/cost of u

    Notes:
    - Unlike Hill Climbing, SAS allows **probabilistic downhill moves**.
    - Cooling schedule: `schedule(t) = 1 / (t^2)`
    '''
    # TODO 3
    u = src
    cost = 0
    p = cost + g.H[u]

    t = 1  # Initial temperature (iteration step)

    while True:
      T = self.schedule(t)  # Cooling schedule
      if T < 1e-5:  # Stop if temperature is too low
        return (u, p)

      neighbors = g.get_neighbors(u)
      if not neighbors:
        return (u, p)  # Return best found state

      # Pick a random neighbor
      neighbor, neighbor_cost = random.choice(neighbors)
      new_p = (cost + neighbor_cost) + g.H[neighbor]

      # Always accept if it's better, otherwise use probability
      delta_E = new_p - p
      if new_p > p or random.uniform(0, 1) < math.exp(delta_E / T):
        u, p = neighbor, new_p
        cost += neighbor_cost

      t += 1  # Increase iteration step


# Evaluation

In [35]:
hcs = HillClimbingSearch()
lbs = LocalBeamSearch()
sas = SimulatedAnnealingSearch()

for stg in [hcs, lbs, sas]:
  print(stg)
  u, p = stg.search(g, 0)
  print(u, p)

5 34
5 34
5 20


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-lec06-LocalSearch-HW.ipynb**
*   Finally, submit the file